In [1]:
import gurobipy as gp
from gurobipy import GRB
import json
from pathlib import Path
import pandas as pd

In [2]:
dimension = "3_30_instances"
folder_projects = Path("instances_projects")
filename_projects = "generic_3_30_projects_0.csv"
filepath_projects = folder_projects / dimension / filename_projects
folder_students = Path("instances_students")
filename_students = "generic_3_30_students_0.csv"
filepath_students = folder_students / dimension / filename_students
projects_info = pd.read_csv(filepath_projects)
students_info = pd.read_csv(filepath_students)
students_info["fav_partners"] = students_info["fav_partners"].apply(json.loads)
students_info["project_prefs"] = students_info["project_prefs"].apply(lambda x: tuple(json.loads(x)))

In [3]:
print(projects_info)

                       name  desired#groups  max#groups  ideal_group_size  \
0         Retail Management               2           4                 2   
1        Management Science               2           5                 4   
2  Personnel Administration               2           5                 2   

   min_group_size  max_group_size  pen_groups  pen_size  
0               1               3           3         2  
1               3               6           2         3  
2               1               3           1         1  


In [ ]:
print(students_info)

                 name  fav_partners project_prefs
0     Donald Thompson   [18, 21, 7]     (2, 0, 0)
1       Kathryn Gomez   [5, 13, 22]     (2, 2, 1)
2    Stephanie Nelson    [5, 13, 1]     (2, 2, 1)
3         Jack Wright  [27, 23, 18]     (2, 1, 3)
4      Carl Gutierrez   [18, 3, 29]     (2, 1, 3)
5      Dorothy Thomas   [1, 18, 29]     (2, 2, 1)
6         Carol Baker  [29, 17, 22]     (2, 3, 1)
7        Walter Myers    [0, 21, 9]     (2, 1, 0)
8         Steven Hill   [17, 7, 14]     (2, 1, 1)
9     Nicholas Martin    [0, 2, 12]     (2, 1, 1)
10   Peter Richardson   [16, 4, 26]     (1, 1, 2)
11      Paul Thompson  [25, 23, 12]     (2, 1, 2)
12  Brittany Castillo   [11, 9, 10]     (2, 1, 1)
13      Kathleen Hill    [1, 2, 19]     (2, 1, 1)
14    Pamela Peterson  [25, 15, 12]     (2, 1, 2)
15         Jose Lopez    [14, 8, 6]     (2, 2, 1)
16     Gabriel Bailey  [19, 17, 24]     (3, 1, 1)
17     Zachary Wright    [6, 22, 7]     (2, 2, 1)
18         Carl Baker    [0, 3, 12]     (2, 1, 2)


In [5]:
reward_bilateral = 2
penalty_unassignment = 3

In [6]:

m = gp.Model()

Set parameter Username
Set parameter LicenseID to value 2654427
Academic license - for non-commercial use only - expires 2026-04-19


In [7]:
project_ids = range(len(projects_info))
student_ids = range(len(students_info))

projects_group_ids = {
    project_id: range(projects_info["max#groups"][project_id])
    for project_id in project_ids
}


In [8]:
x = m.addVars(
    (
        (project_id, group_id, student_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        for student_id in student_ids
    ),
    vtype=GRB.BINARY,
    name="assign",
)
m.update()

In [9]:
y = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    vtype=GRB.BINARY,
    name="establish_group",
)

In [10]:
favorite_partners_students = students_info["fav_partners"].tolist()
mutual_pairs = set()

for student_id, favorite_partners in enumerate(favorite_partners_students):
    for partner_id in favorite_partners:
        if partner_id <= student_id:
            continue
        if student_id in favorite_partners_students[partner_id]:
            mutual_pairs.add((student_id, partner_id))

In [11]:
z = m.addVars(mutual_pairs, vtype=GRB.BINARY, name="not_realize_bilateral_cooperation_wish")

In [12]:
v = m.addVars(student_ids, name = "student_unassigned")

In [13]:
gs_surplus = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_surplus",
)

In [14]:
gs_deficit = m.addVars(
    (
        (project_id, group_id)
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ),
    name="group_size_deficit",
)

In [15]:
project_preferences = {
    (student_id, project_id): students_info["project_prefs"][student_id][project_id]
    for student_id in student_ids
    for project_id in project_ids
}

In [16]:
rewards_student_project_preference = gp.quicksum(
    project_preferences[student_id, project_id] * x[project_id, group_id, student_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    for student_id in student_ids
)

rewards_bilateral_cooperation_wish_realized = reward_bilateral * gp.quicksum(
    1 - z[*mutual_pair] for mutual_pair in mutual_pairs
)

penalties_student_unassigned = penalty_unassignment * gp.quicksum(v.values())

penalties_more_groups_than_offered = gp.quicksum(
    projects_info["pen_groups"][project_id] * y[project_id, group_id]
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
    if group_id >= projects_info["desired#groups"][project_id]
)

penalties_not_ideal_group_size = gp.quicksum(
    projects_info["pen_size"][project_id]
    * (gs_surplus[project_id, group_id] + gs_deficit[project_id, group_id])
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
)

m.setObjective(
    rewards_student_project_preference
    + rewards_bilateral_cooperation_wish_realized
    - penalties_student_unassigned
    - penalties_more_groups_than_offered
    - penalties_not_ideal_group_size,
    sense=GRB.MAXIMIZE,
)

In [17]:
m.addConstrs(
    (x.sum("*", "*", student_id) + v[student_id] == 1 for student_id in student_ids),
    name="penalty_if_unassigned",
)
m.update()

In [18]:
m.addConstrs(
    (
        y[project_id, group_id] <= y[project_id, group_id - 1]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
        if group_id > 0
    ), name="only_consecutive_group_ids"
)
m.update()

In [19]:
m.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        >= projects_info["min_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_min_group_size"
)
m.update()

In [20]:
m.addConstrs(
    (
        x.sum(project_id, group_id, "*")
        <= projects_info["max_group_size"][project_id] * y[project_id, group_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="cap_group_size_at_max"
)
m.update()

In [21]:
m.addConstrs(
    (
        gs_surplus[project_id, group_id]
        >= x.sum(project_id, group_id, "*")
        - projects_info["ideal_group_size"][project_id]
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_surplus"
)
m.update()

In [22]:
m.addConstrs(
    (
        gs_deficit[project_id, group_id]
        >= projects_info["ideal_group_size"][project_id]
        - x.sum(project_id, group_id, "*")
        - projects_info["max_group_size"][project_id] * (1 - y[project_id, group_id])
        for project_id in project_ids
        for group_id in projects_group_ids[project_id]
    ), name="ensure_correct_group_size_deficit"
)
m.update()

In [23]:
max_num_groups = max(projects_info["max#groups"])


unique_group_identifiers = {
    (project_id, group_id): project_id + group_id / max_num_groups
    for project_id in project_ids
    for group_id in projects_group_ids[project_id]
}

In [24]:
num_projects = len(projects_info)

m.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, first_student_id]
                - x[project_id, group_id, second_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_1"
)
m.update()

In [25]:
m.addConstrs(
    (
        z[first_student_id, second_student_id] * num_projects
        >= sum(
            unique_group_identifiers[project_id, group_id]
            * (
                x[project_id, group_id, second_student_id]
                - x[project_id, group_id, first_student_id]
            )
            for project_id in project_ids
            for group_id in projects_group_ids[project_id]
        )
        for (first_student_id, second_student_id) in mutual_pairs
    ), name="ensure_correct_inidicator_different_group_2"
)
m.update()

In [26]:
m.Params.TimeLimit = 60
m.optimize()

Set parameter TimeLimit to value 60
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-12500H, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  60

Optimize a model with 143 rows, 515 columns and 3464 nonzeros
Model fingerprint: 0x45ef2a53
Variable types: 58 continuous, 457 integer (457 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective -44.0000000
Presolve removed 14 rows and 0 columns
Presolve time: 0.01s
Presolved: 129 rows, 515 columns, 3044 nonzeros
Variable types: 0 continuous, 515 integer (487 binary)

Root relaxation: objective 9.238185e+01, 375 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work

In [27]:
for project_id in project_ids:
    print("\n")
    print(projects_info["name"][project_id])
    for group_id in projects_group_ids[project_id]:
        print("\nGroup", group_id + 1)
        for student_id in student_ids:
            if x[project_id, group_id, student_id].X > 0.5:
                print(students_info["name"][student_id], student_id)



Retail Management

Group 1
Steven Hill 8
Gabriel Bailey 16

Group 2
Donald Thompson 0
Walter Myers 7
Lawrence Hall 21

Group 3

Group 4


Management Science

Group 1
Stephanie Nelson 2
Pamela Peterson 14
Jose Lopez 15
Janet Scott 25

Group 2
Kathryn Gomez 1
Dorothy Thomas 5
Kathleen Hill 13
Cynthia Thompson 19

Group 3
Carol Baker 6
Zachary Wright 17
Charles Thomas 22
Jeremy Baker 29

Group 4
Nicholas Martin 9
Paul Thompson 11
Brittany Castillo 12
Jacob Williams 23

Group 5


Personnel Administration

Group 1
Carl Gutierrez 4
Teresa Hall 20

Group 2
Jack Wright 3
Carl Baker 18
Amy Evans 27

Group 3
Elizabeth Collins 24
Lisa Thomas 28

Group 4
Peter Richardson 10
Andrew Bailey 26

Group 5


In [28]:
# m.write("SSPAGDP.lp")

In [29]:
m.dispose()
gp.disposeDefaultEnv()

Freeing default Gurobi environment
